# GRMP Attack Experiment - Google Colab

This notebook runs the Graph Representation-based Model Poisoning (GRMP) attack experiment on AG News dataset.

**Paper**: Graph Representation-based Model Poisoning on the Heterogeneous Internet of Agents

## Setup Instructions

1. **Enable GPU**: Runtime → Change runtime type → GPU
2. **Fetch Code**: Run **Step 0** to clone/download the repo if only this notebook was uploaded.
3. **Run all cells**: Runtime → Run all
4. **View results**: Check the `results/` folder for outputs and visualizations



## Step 0: Fetch Code
If you only uploaded this notebook, run this to clone the repository and set the working directory.
If you've already uploaded the Python files, it will reuse them without cloning.


In [ ]:
# Fetch repository and set working directory
import os, sys, subprocess
from pathlib import Path

REPO_URL = 'https://github.com/GuangLun2000/IoA-Attack-GRMP.git'
REPO_DIR = Path('IoA-Attack-GRMP')

def code_files_present():
    return Path('main.py').exists() and Path('client.py').exists()

if code_files_present():
    print('✅ Code files found in current directory.')
else:
    if REPO_DIR.exists():
        print(f'🔁 Using existing folder: {REPO_DIR}')
    else:
        print(f'📥 Cloning {REPO_URL} ...')
        subprocess.run(['git', 'clone', '--depth', '1', REPO_URL], check=True)
    os.chdir(REPO_DIR)
    print(f"✅ Switched to {Path('.').resolve()}")

# Ensure current path is importable for subsequent cells
sys.path.append(str(Path('.').resolve()))
print(f"📂 Working directory: {Path('.').resolve()}")



## Step 1: Install Dependencies


In [ ]:
# Install required packages
from pathlib import Path
req = Path('requirements.txt')
if req.exists():
    print('Installing from requirements.txt ...')
    %pip install -q -r requirements.txt
else:
    print('requirements.txt not found; installing explicit package list...')
    %pip install -q torch>=2.0.0 transformers>=4.35.0 datasets>=2.0.0 numpy>=1.21.0 scikit-learn>=1.0.0 pandas>=1.3.0 tqdm>=4.62.0 matplotlib>=3.4.0 seaborn>=0.11.0

print('✅ Dependencies installed successfully!')


## Step 2: Verify Files and GPU


In [ ]:
# Check if files exist
import os
from pathlib import Path

required_files = ['main.py', 'client.py', 'server.py', 'data_loader.py', 'models.py', 'visualization.py']
missing_files = [f for f in required_files if not os.path.exists(f)]

if missing_files:
    print(f"⚠️  Missing files: {missing_files}")
    print("Please upload these files to Colab using the file uploader.")
else:
    print("✅ All required files found!")
    for f in required_files:
        print(f"  - {f}")

# Check GPU
import torch
print(f"\nPyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU Device: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️  No GPU detected. Training will be slower.")
    print("   Go to Runtime → Change runtime type → GPU")


## Step 3: Run Experiment

**Note**: All experiment parameters are configured in `main.py`. 
To modify parameters, edit the `config` dictionary in `main.py` before running this cell.


In [ ]:
# All experiment parameters are configured in main.py
# The experiment will use the config dictionary defined in main.py's main() function
print("✅ Using configuration from main.py")
print("   To modify parameters, edit the 'config' dictionary in main.py")

## Step 4: Run Experiment

The experiment will use the configuration from `main.py`. 
All parameters are defined in the `config` dictionary in `main.py`'s `main()` function.


In [ ]:
# Run the experiment using configuration from main.py
import sys
import warnings
warnings.filterwarnings('ignore')

# Import and run main() function which contains all configuration
from main import main

print("🚀 Starting GRMP Attack Experiment...")
print("=" * 60)
print("Using configuration from main.py")
print("=" * 60)

try:
    # main() function will handle everything: setup, run, and visualization
    main()
    
    print("\n✅ Experiment completed successfully!")
except Exception as e:
    print(f"\n❌ Experiment failed: {e}")
    import traceback
    traceback.print_exc()


## Step 5: View Results and Visualizations


In [ ]:
# Display visualization plots
from IPython.display import Image, display
from pathlib import Path

results_dir = Path("results")
# Use default experiment name from main.py (can be changed in main.py config)
experiment_name = 'vgae_grmp_attack'

# List of figures to display
figures = [
    ("Figure 3: Global Accuracy and ASR", f"{experiment_name}_figure3.png"),
    ("Figure 4: Cosine Similarity", f"{experiment_name}_figure4.png"),
    ("Figure 5: Local Accuracy (No Attack)", f"{experiment_name}_figure5.png"),
    ("Figure 6: Local Accuracy (With Attack)", f"{experiment_name}_figure6.png"),
]

print("📊 Displaying Visualization Figures:")
print("=" * 60)

for fig_title, fig_name in figures:
    fig_path = results_dir / fig_name
    if fig_path.exists():
        print(f"\n✅ {fig_title}")
        display(Image(str(fig_path)))
    else:
        print(f"\n⚠️  {fig_title} not found")


In [ ]:
# Load and display experiment results summary
import json
from pathlib import Path

results_dir = Path("results")
# Use default experiment name from main.py (can be changed in main.py config)
experiment_name = 'vgae_grmp_attack'
results_path = results_dir / f"{experiment_name}_results.json"

if results_path.exists():
    with open(results_path, 'r') as f:
        results_data = json.load(f)
    
    print("📊 Experiment Results Summary:")
    print("=" * 60)
    
    # Display key metrics
    rounds = results_data['progressive_metrics']['rounds']
    clean_acc = results_data['progressive_metrics']['clean_acc']
    attack_asr = results_data['progressive_metrics']['attack_asr']
    
    print(f"\nTotal Rounds: {len(rounds)}")
    print(f"Final Clean Accuracy: {clean_acc[-1]:.4f}")
    print(f"Final Attack Success Rate (ASR): {attack_asr[-1]:.4f}")
    print(f"Peak ASR: {max(attack_asr):.4f}")
    
    # Display per-round summary
    print("\n📈 Per-Round Summary:")
    print("Round | Clean Acc | ASR")
    print("-" * 30)
    for i, (r, acc, asr) in enumerate(zip(rounds, clean_acc, attack_asr)):
        if i % 5 == 0 or i == len(rounds) - 1:  # Show every 5th round and last round
            print(f"{r:5d} | {acc:9.4f} | {asr:.4f}")
    
    # Display local accuracies if available
    if 'local_accuracies' in results_data and results_data['local_accuracies']:
        print("\n📊 Local Accuracies (Last Round):")
        local_accs = results_data['local_accuracies']
        for client_id, accs in sorted(local_accs.items()):
            if accs:
                print(f"  Client {client_id}: {accs[-1]:.4f}")
else:
    print(f"⚠️  Results file not found: {results_path}")


## Step 6: Download Results


In [ ]:
# Create a zip file with all results
import zipfile
from pathlib import Path

results_dir = Path("results")
zip_path = "grmp_experiment_results.zip"

if results_dir.exists():
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for file_path in results_dir.rglob('*'):
            if file_path.is_file():
                zipf.write(file_path, file_path.relative_to(results_dir.parent))
    
    print(f"✅ Created zip file: {zip_path}")
    print(f"\n📥 Download the file using the cell below")
else:
    print("⚠️  Results directory not found.")


In [ ]:
# Download results zip file
from google.colab import files

if Path("grmp_experiment_results.zip").exists():
    files.download('grmp_experiment_results.zip')
    print("✅ Download started!")
else:
    print("⚠️  Zip file not found. Run the previous cell first.")

## Step 6: Exit Running Time


In [ ]:
from google.colab import runtime
import time

print("✅ All done. 所有任务运行完毕。")

time.sleep(2)

runtime.unassign()